- 임베딩한다 = '고차원으로 보낸다'

# Word2vec
- '유사한 문맥에 등장한 단어는 유사한 의미를 갖는다' 라는 기정으로 벡터화!
- CBOW: 주변 단어로 '중간에 있는 단어' 예측
- Skip-Gram: 중심 단어로 '주변단어' 예측
- 단어간의 유의미한 관계를 잘 측정할 수 있다!(단어간 유사도)
- https://word2vec.kr/search/ 여기서 워드투백 맛보기!

Similarity
- 총 5가지 정도 있음(cosine, jaccard만 봐보자)
- cosine similarity: 각도가 작은게 제일 비슷한거임
- jaccard similarity: True/False 형태로 구분할 수 있는 경우에 좋다(좋아요, 구매여부 등)


## Cosine Similarity

In [2]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
x = np.array([1,2,3]).reshape(1, -1)
y = np.array([4,5,6]).reshape(1, -1)
z = np.array([-10,-20,-30]).reshape(1, -1)

In [8]:
x.shape

(1, 3)

In [5]:
cosine_similarity(x, y)

array([[0.97463185]])

In [6]:
cosine_similarity(y, z)

array([[-0.97463185]])

In [7]:
cosine_similarity(x, z)

array([[-1.]])

## Jaccard Similarity

In [13]:
set1 = {'choi', 'jin', 'yeong'}
set2 = {'choi', 'jin', 'woong'}

intersection = set1.intersection(set2) # set1&set2
print(intersection)
union = set1.union(set2) # set1|set2
print(union)

{'choi', 'jin'}
{'jin', 'choi', 'yeong', 'woong'}


In [14]:
jaccard_sim = len(intersection)/len(union)
print(jaccard_sim)

0.5


In [15]:
from sklearn.metrics import jaccard_score

vec1 = [1,1,1,0,0]
vec2 = [1,0,1,0,0]
jaccard_score(vec1, vec2)

0.6666666666666666

# 추천시스템

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
movies = pd.read_csv('/content/drive/MyDrive/8_DL/movies_metadata.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [18]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [19]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [20]:
movies.shape

(45466, 24)

In [21]:
movies = movies.head(20000) # 메모리 에러가 나는 경우 - 코랩

In [22]:
movies['overview'].isnull().sum()

135

In [23]:
movies = movies.dropna(subset = ['overview'])

In [24]:
movies['overview'].isnull().sum()

0

In [25]:
tfidf_vec = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf_vec.fit_transform(movies['overview'])

In [26]:
tfidf_matrix.shape

(19865, 47487)

In [27]:
# 벡터화 전
movies['overview'][0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [28]:
# 벡터화 후
tfidf_matrix.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [29]:
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [30]:
cosine_sim_matrix.shape # comlumn개수가 row개수랑 같아짐

(19865, 19865)

In [32]:
movies_titles = movies['original_title'].values
print(len(movies_titles))
movies_titles

19865


array(['Toy Story', 'Jumanji', 'Grumpier Old Men', ..., 'Two in the Wave',
       'Lotte Reiniger: Homage to the Inventor of the Silhouette Film',
       "RKO Production 601: The Making of 'Kong, the Eighth Wonder of the World'"],
      dtype=object)

In [33]:
cos_sim_df = pd.DataFrame(cosine_sim_matrix, columns = movies_titles, index = movies_titles)
cos_sim_df.head()

# 영화 줄거리를 벡터화 시켜서.. 영화끼리의 유사도를 측정

,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Il ne faut jurer... de rien !,Calmos,How to Make Love to a Woman,"After Fall, Winter",Violeta se fue a los cielos,L'Ordre et la Morale,Versailles,Two in the Wave,Lotte Reiniger: Homage to the Inventor of the Silhouette Film,"RKO Production 601: The Making of 'Kong, the Eighth Wonder of the World'"
Toy Story,1.000000,0.015752,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.009968,0.060988,0.000000,0.0,0.0,0.024569,0.0,0.0,0.0
Jumanji,0.015752,1.000000,0.049069,0.0,0.000000,0.051801,0.000000,0.000000,0.106296,0.0,...,0.029225,0.000000,0.000000,0.004263,0.0,0.0,0.000000,0.0,0.0,0.0
Grumpier Old Men,0.000000,0.049069,1.000000,0.0,0.024997,0.000000,0.000000,0.006487,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
Waiting to Exhale,0.000000,0.000000,0.000000,1.0,0.000000,0.007122,0.000000,0.009382,0.000000,0.0,...,0.008295,0.000000,0.008937,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
Father of the Bride Part II,0.000000,0.000000,0.024997,0.0,1.000000,0.000000,0.032964,0.000000,0.032720,0.0,...,0.009268,0.000000,0.000000,0.025405,0.0,0.0,0.078383,0.0,0.0,0.0


In [35]:
cos_sim_df['Toy Story'].sort_values(ascending = False)

Toy Story                                                                   1.000000
Toy Story 3                                                                 0.525903
Toy Story 2                                                                 0.462683
The 40 Year Old Virgin                                                      0.279532
The Champ                                                                   0.200642
                                                                              ...   
New York Minute                                                             0.000000
Van Helsing                                                                 0.000000
The Mudge Boy                                                               0.000000
Valentín                                                                    0.000000
RKO Production 601: The Making of 'Kong, the Eighth Wonder of the World'    0.000000
Name: Toy Story, Length: 19865, dtype: float64